In [1]:
import sys
import os
import importlib
import time
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer #计算tfidf
from sklearn.feature_extraction.text import CountVectorizer  #计算df
from sklearn.feature_extraction.text import TfidfVectorizer  #“一步到位”
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn import metrics

id_num = []
word_article = []
words_article = []
label = []
X_train = []
y_train = []
X_test = []
y_test = []

load_start = time.time()
importlib.reload(sys)
with open('train_set.csv','r',encoding = 'utf-8') as f:
    lines = f.readlines()
    for line in lines[1:]:
        line = line.split(',')
        X_train.append(line[2])
        y_train.append(line[3])
with open('test_set.csv','r',encoding = 'utf-8') as f:
    lines = f.readlines()
    for line in lines[1:]:
        line = line.split(',')
        X_test.append(line[2])
print("Time consumption on loading:",time.time() - load_start)
"""
step = [
            ('vect',CountVectorizer()),
            ('tfidf',TfidfTransformer()),
#             ('pca',PCA(n_components=3000)),
#             ('clf',RandomForestClassifier()),
#             ('clf',svm.SVC(C=1,kernel='linear'))
            ('clf',svm.LinearSVC(C=0.3))
#             ('clf',tree.DecisionTreeClassifier())
        ]
ppl_clf = Pipeline(step)
"""
train_start = time.time()
vect = TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1, smooth_idf=1, sublinear_tf=1)
vect.fit(X_train) # 或者直接X_train = vect.fit_transform(X_train)
X_train = vect.transform(X_train)
# ppl_clf = svm.LinearSVC(C=1)
# ppl_clf = svm.SVC(kernel='linear', C=0.8, )
ppl_clf = SGDClassifier(loss="modified_huber", penalty="elasticnet", alpha=0.00001, n_jobs=-1)
ppl_clf.fit(X_train, y_train)
# joblib.dump(ppl_clf, 'lineSVM_20000__.m')
del X_train,y_train #释放内存
print("Time consumption on training:",time.time() - train_start)

test_start = time.time()
X_test = vect.transform(X_test)
# ppl_clf = joblib.load('lineSVM_20000__.m')
prediction = ppl_clf.predict(X_test)
f_out = open('ans.csv','w')
f_out.write("id,class"+"\n")
for i in range(X_test.shape[0]): #是列数，即句子数
    f_out.write(str(i)+","+str(prediction[i]))
f_out.close()
# print(metrics.classification_report(y_test,prediction))
# print(metrics.confusion_matrix(y_test,prediction))
print("Time consumption on testing:",time.time() - test_start)

Time consumption on loading: 4.979656457901001


/home/amax/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Time consumption on training: 869.7680995464325
Time consumption on testing: 253.66622519493103
